# Step 2: Data Cleaning

## A: Load the Necessary Libraries & Files

In [ ]:
# Libraries 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

In [2]:
# Files
customers_df = pd.read_csv('customers.csv')
geolocation_df = pd.read_csv('geolocation.csv')
order_items_df = pd.read_csv('order_items.csv')
orders_df = pd.read_csv('orders.csv')
payments_df = pd.read_csv('payments.csv')
products_df = pd.read_csv('products.csv')
sellers_df = pd.read_csv('sellers.csv')

## B: Test for Missing Data

### i. The customers table

In [5]:
customers_df.isnull().sum()

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

### ii. The geolocation table

In [6]:
geolocation_df.isnull().sum()

geolocation_zip_code_prefix    0
geolocation_lat                0
geolocation_lng                0
geolocation_city               0
geolocation_state              0
dtype: int64

### iii. The order_items table


In [7]:
order_items_df.isnull().sum()

order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

### iv. The orders table

In [12]:
orders_df.isnull().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

In [14]:
orders_df.shape

# So, it's only ~2-3% of delivered_dates that are missing data.

(99441, 8)

### v. The payments table

In [9]:
payments_df.isnull().sum()

order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64

### vi. The prouducts table

In [11]:
products_df.isnull().sum()

product_id                      0
product category              610
product_name_length           610
product_description_length    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64

In [ ]:
products_df.shape

# less than 2% of data missing from category, name, description columns. 

(32951, 9)

### vii. The sellers table

In [16]:
sellers_df.isnull().sum()

seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
dtype: int64

### Takeaways: 
* Most tables, surprisingly, weren't missing any data
* Only the orders and products tables were missing any data at all, and then it was only ~2-3% of a handful of columns
* The columns in question may not even be relevant to the analysis (i.e. may be dropped anyways)

## C. Test for Irregular Data

### i. The customers table

In [ ]:
customers_df.head(1)

# No numeric columns



,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP


### ii. The geolocation table

In [ ]:
geolocation_df.head(1)

# No numeric data

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP


### iii. The order_items table

In [ ]:
order_items_df.head(1)

# Could look at the price and freight_value columns

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29
